# Lecture 13
## Non Linear Optimization with JuMP (Part 2)
## Date: 28.11

## Constrained Non Linear Optimization

In order to solve constrained non linear problems we can leverage ```JuMP``` (as we did for linear problems) using non linear solvers such as the ```Ipopt```.

### Example 

Take the following constrained minimization problem:
$$
\text{min } (x_1 - 3)^3 + (x_2 - 4)^2 \text{ s.t. } \begin{cases} (x_1 - 1)^2 + (x_2 + 1)^3 + e^{-x_1} \leq 1 \end{cases}
$$

In ```JuMP``` we can express it as:

In [1]:
using JuMP, Ipopt;

In [2]:
m = Model(with_optimizer(Ipopt.Optimizer));

In [3]:
@variable(m, x[1:2])

2-element Array{VariableRef,1}:
 x[1]
 x[2]

In [4]:
@NLobjective(m, Min, (x[1]-3)^3 + (x[2]-4)^2)

In [5]:
@NLconstraint(m, (x[1]-1)^2 + (x[2]+1)^3 + exp(-x[1]) <= 1)

((x[1] - 1.0) ^ 2.0 + (x[2] + 1.0) ^ 3.0 + exp(-x[1])) - 1.0 ≤ 0

In [6]:
JuMP.optimize!(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [7]:
println("** Optimal objective function value = ", JuMP.objective_value(m))
println("** Optimal solution = ", JuMP.value.(x))

** Optimal objective function value = 4.409110764366554
** Optimal solution = [0.492399, -0.491889]


Another awesome, however more complex, example is given at the [juliaopt webpage](https://www.juliaopt.org/notebooks/JuMP-Rocket.html)

### Exercise

A worker can exert two effort levels, high or low, which induce a production error with probability 0.25 and 0.75 respectively. His utility function id $U(w,e) = 100 - \frac{10}{w} - v$, where $w$ is the wage and $v$ takes the value of 2 if effort is high and 0 if effort is low. Production errors are observable and so can be introduced into the worker's contract, but effort cannot. The product obtained is worth 20 if there are no errors and 0 otherwise. The principal is risk neutral. Assume the worker has reservation utility equal to $\bar{U} = 0$. Calculate the optimal contract and the effort that the principal desires, under condition of symmetric information.

### Exercise

In [8]:
model = Model(with_optimizer(Ipopt.Optimizer));

In [9]:
xₕ = 20;
xₗ = 0;

In [10]:
@variable(model, wₗ >= 0, start = 0.0);
@variable(model, wₕ >= 0, start = 0.0);

@variable(model, 0<=e<=1, start = 0.0);

In [11]:
@NLobjective(model, Max, e*(0.75(xₕ - wₕ) + 0.25(xₗ - wₗ) + (1-e)*(0.25(xₕ - wₕ) + 0.75(xₗ - wₗ))))

In [12]:
#Think economically why this constraint is binding
@NLconstraint(model, e*(0.75(100 - 10/wₕ - 2) + 0.25(100 - 10/wₗ - 2)) + (1-e)*(0.25(100 - 10/wₕ - 0) + 0.75(100 - 10/wₗ - 0)) == 0);

In [13]:
model

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Nonlinear: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: e, wₕ, wₗ

In [14]:
JuMP.optimize!(model)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [15]:
println("** Optimal profits = ", JuMP.objective_value(model))
println("** Optimal low outcome wage = ", JuMP.value.(wₗ))
println("** Optimal high outcome wage = ", JuMP.value.(wₕ))
println("** Optimal effort = ", JuMP.value.(e))

** Optimal profits = 14.89795928114674
** Optimal low outcome wage = 0.10204081885273428
** Optimal high outcome wage = 0.10204081551153305
** Optimal effort = 1.0


Did you noticed something interesting?

### What about Mixed Integer?

In this case you have two ways:

* Using solvers already provided by the ```JuMP``` package such as:
 * Juniper;
 * SCIP.
* Using solvers that support the ```.nl``` format by means of the ```AmplNLWriter```.

#### An example using AmplNLWriter

First thing first you need to install the ```AmplNLWriter``` package into Julia.

In [16]:
using AmplNLWriter

Then you need to get the source code from the [AMPL page](https://ampl.com/products/solvers/open-source/).

Save it in whatever directory you like, I the ```bonmin``` solver in ```~/Solvers/bonmin``` 

In [17]:
m = Model(with_optimizer(AmplNLWriter.Optimizer, "/home/mrepetto94/Solvers/bonmin/bonmin"));

In [18]:
@variable(m, x[1:2], Int)

2-element Array{VariableRef,1}:
 x[1]
 x[2]

In [19]:
@NLobjective(m, Min, (x[1]-3)^3 + (x[2]-4)^2)

In [20]:
@NLconstraint(m, (x[1]-1)^2 + (x[2]+1)^3 + exp(-x[1]) <= 1)

((x[1] - 1.0) ^ 2.0 + (x[2] + 1.0) ^ 3.0 + exp(-x[1])) - 1.0 ≤ 0

In [21]:
JuMP.optimize!(m)

Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 4.4091108        7 0.002301
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1      INFEAS 0.33907697       24 0.007433
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see de

In [22]:
println("** Optimal objective function value = ", JuMP.objective_value(m))
println("** Optimal solution = ", JuMP.value.(x))

** Optimal objective function value = -1052.0
** Optimal solution = [-9.0, -22.0]


### Exercise

Try to achieve the same result but this time using a solver already provided by ```JuMP```.

### One last exercise

A worker can exert two effort levels, high or low, which induce a production error with probability 0.25 and 0.75 respectively. His utility function id $U(w,e) = 100 - \frac{10}{w} - v$, where $w$ is the wage and $v$ takes the value of 2 if effort is high and 0 if effort is low. Production errors are observable and so can be introduced into the worker's contract, but effort cannot. The product obtained is worth 20 if there are no errors and 0 otherwise. The principal is risk neutral. Assume the worker has reservation utility equal to $\bar{U} = 0$. Calculate the optimal contract and the effort that the principal desires, under condition of **asymmetric information** on the agent's behavior.

**Hint:** in asymmetric information the principal offers two types of contracts since the only thing it can see is the outcome.

### Solution

In [23]:
model = Model(with_optimizer(Ipopt.Optimizer));

In [24]:
xₕ = 20;
xₗ = 0;

In [25]:
@variable(model, wₗ >= 0, start = 0.0);
@variable(model, wₕ >= 0, start = 0.0);

In [26]:
#I want to maximize only for the high type 
@NLobjective(model, Max, (0.75(xₕ - wₕ)+ 0.25(xₗ - wₗ)))

In [27]:
#Participation constraint -high type- 
@NLconstraint(model, (0.75(100 - 10/wₕ - 2) + 0.25(100 - 10/wₗ - 2)) == 0);

#Incentive compatibility -high type-
@NLconstraint(model, (0.75(100 - 10/wₕ - 2) + 0.25(100 - 10/wₗ - 2)) >= (0.25(100 - 10/wₕ - 0) + 0.75(100 - 10/wₗ - 0)));

In [28]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Nonlinear: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: wₕ, wₗ

In [29]:
JuMP.optimize!(model);

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [30]:
println("** Optimal profits = ", JuMP.objective_value(model))
println("** Optimal low wage = ", JuMP.value.(wₗ))
println("** Optimal high wage = ", JuMP.value.(wₕ))

** Optimal profits = 14.897927934599176
** Optimal low wage = 0.09900978006012431
** Optimal high wage = 0.10309282718105858


In [31]:
JuMP.value.(wₕ) > JuMP.value.(wₗ)

true

In this setting Asymmetric information was a good or bad thing for the firm?